In [1]:
!pip install python-dotenv bs4 google-generativeai ipython openai

#
# # Full code
#

# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import google.generativeai as genai
import gradio as gr


# Get secret key from huggingface space settings
# Print the key prefixes to help with any debugging

openai_api_key = os.environ['OPENAI_API_KEY']
#anthropic_api_key = os.environ['ANTHROPIC_API_KEY']
google_api_key = os.environ['GOOGLE_API_KEY']

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

#if anthropic_api_key:
#   print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
#else:
#    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")


# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()
#claude = anthropic.Anthropic()
genai.configure(api_key=google_api_key)

# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

# system prompt for getting links from website

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

# user prompt for getting links from website

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

def get_links(url,llm,modelname,temperature):
    website = Website(url)

    print("llm =",llm)
    print("modelname =",modelname)

    match llm:
        case 'openai':
            response = openai.chat.completions.create(
                model=modelname,
                temperature=temperature,
                messages=[
                    {"role": "system", "content": link_system_prompt},
                    {"role": "user", "content": get_links_user_prompt(website)}
            ],
                response_format={"type": "json_object"}
            )
            result = response.choices[0].message.content
        case 'googleai':
            model=genai.GenerativeModel(model_name=modelname,system_instruction=link_system_prompt)
            response = model.generate_content(get_links_user_prompt(website),generation_config=genai.GenerationConfig(temperature=temperature,response_mime_type="application/json"))
            result = response.text

    return json.loads(result)

def get_all_details(url,llm,modelname,temperature):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url,llm,modelname,temperature)
    #print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result



# system prompt for AI to create brochure


# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short humorous, entertaining brochure about the company for prospective and previous customers.
Include details of company culture and customers if you have the information. Focus on work to create or maintain pickle ball courts.
Write in a theme for Valentine's Day.
Emphasize customers that may need upkeep and maintenance of their existing property and installations.
Return results in Markdown.
"""

# Create user prompt with website details included

def get_brochure_user_prompt(company_name,url,llm,modelname,temperature):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url,llm,modelname,temperature)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

#
# # Everything below here uses values from the gradio web interface. Not the static values defined above
#
def stream_openai(system_prompt,company_name,url,llm,modelname,temperature):
    response = openai.chat.completions.create(
        model=modelname,
        stream=True,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name,url,llm,modelname,temperature)}
        ],
    )

    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

    print(result)


def stream_googleai(system_prompt,company_name,url,llm,modelname,temperature):
    model=genai.GenerativeModel(
        model_name=modelname,
        system_instruction=system_prompt)
    responses = model.generate_content(
        get_brochure_user_prompt(company_name,url,llm,modelname,temperature),
        generation_config = genai.GenerationConfig(
            temperature=temperature
        ),
        stream=True
    )

    result = ""
    for response in responses:
        result += response.text
        yield result

    print(result)


def stream_model(system_prompt,company_name,url,llm_model,temperature):

    print("Analyzing %s from %s" % (company_name,url))

    match llm_model:
        case 'openai_gpt-4o-mini':
            llm = "openai"
            modelname = "gpt-4o-mini"
            print("\n*** Using LLM %s and modelname %s\n" % (llm,modelname))
            result = stream_openai(system_prompt,company_name,url,llm,modelname,temperature)
        case 'googleai_gemini-1.5-pro':
            llm = "googleai"
            modelname = "gemini-1.5-pro"
            print("\n***Using LLM %s and modelname %s\n" % (llm,modelname))
            result = stream_googleai(system_prompt,company_name,url,llm,modelname,temperature)
        case 'googleai_gemini-1.5-flash':
            llm = "googleai"
            modelname = "gemini-1.5-flash"
            print("\n***Using LLM %s and modelname %s\n" % (llm,modelname))
            result = stream_googleai(system_prompt,company_name,url,llm,modelname,temperature)
        case _:
            raise ValueError("Unknown LLM and MODEL combination")

    yield from result


view = gr.Interface(
    fn=stream_model,
    inputs=[
        gr.Textbox(label="AI Agent",value=system_prompt),
        gr.Textbox(label="Company Name", value="Saviano, Co."),
        gr.Textbox(label="Website to analyze", value="https://saviano.com"),
        gr.Dropdown(["openai_gpt-4o-mini", "googleai_gemini-1.5-pro", "googleai_gemini-1.5-flash"], label="Select AI Engine and LLM Model", value="googleai_gemini-1.5-pro"),
        gr.Slider(0,2,value=0.7,label="AI Temperature",info="AI Creativity Level (higher = more creative)")
    ],
    outputs=[
        gr.Markdown(label="Response:")
    ],
    flagging_mode="never"
)

view.launch()

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyDi
* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Analyzing Saviano, Co. from https://saviano.com

***Using LLM googleai and modelname gemini-1.5-pro

llm = googleai
modelname = gemini-1.5-pro


Traceback (most recent call last):
  File "/Users/adang4/Documents/devwork/andersondang/llm_engineering/llms/lib/python3.11/site-packages/google/api_core/grpc_helpers.py", line 76, in error_remapped_callable
    return callable_(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/adang4/Documents/devwork/andersondang/llm_engineering/llms/lib/python3.11/site-packages/grpc/_interceptor.py", line 277, in __call__
    response, ignored_call = self._with_call(
                             ^^^^^^^^^^^^^^^^
  File "/Users/adang4/Documents/devwork/andersondang/llm_engineering/llms/lib/python3.11/site-packages/grpc/_interceptor.py", line 332, in _with_call
    return call.result(), call
           ^^^^^^^^^^^^^
  File "/Users/adang4/Documents/devwork/andersondang/llm_engineering/llms/lib/python3.11/site-packages/grpc/_channel.py", line 440, in result
    raise self
  File "/Users/adang4/Documents/devwork/andersondang/llm_engineering/llms/lib/python3.11/site-packages/grpc/_inte

Analyzing Saviano, Co. from https://saviano.com

***Using LLM googleai and modelname gemini-1.5-pro

llm = googleai
modelname = gemini-1.5-pro
## Love is in the Air (and on the Court!) ❤️  Saviano Co.'s Valentine's Day Pickleball Pampering

This Valentine's Day, don't let your pickleball court feel neglected! Show it some love with Saviano Co., the experts in court construction and maintenance since 1963.  We're serving up satisfaction (and aces!) to clients nationally and globally.

**Is your court feeling a little... *cracked*?**  💔

Don't let those fissures in your playing surface become a relationship dealbreaker.  We'll resurface your court with the tenderness of a first date, leaving it smooth and ready for action.  From paving and grading to drainage and fencing, we handle every detail with the utmost care, just like your true love deserves.

**Tired of that *faded* look?**  😞

We'll give your court a Valentine's Day makeover it will never forget! Our expert resurfacing services